In [ ]:
!git clone https://github.com/ultralytics/yolov5  

Cloning into 'yolov5'...
remote: Enumerating objects: 10069, done.
remote: Total 10069 (delta 0), reused 0 (delta 0), pack-reused 10069
Receiving objects: 100% (10069/10069), 10.36 MiB | 7.02 MiB/s, done.
Resolving deltas: 100% (6986/6986), done.


In [ ]:
!pip install roboflow